In [1]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash('HW4', external_stylesheets=external_stylesheets)

soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=distinct spc_common').replace(' ', '%20')
all_trees = pd.read_json(soql_url)

app.layout = html.Div(
    children=[
        html.Div(dcc.Dropdown(
                options=all_trees['spc_common'][all_trees['spc_common'].notnull()].tolist(), 
                value='red maple', 
                id='tree-dropdown',
                className="four columns"
                ),className="row"),
    html.Div(id='plot'),
])

@app.callback(Output('plot','children'),[Input('tree-dropdown', 'value')])
def callback(value):
    soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,health,count(*)' +\
        '&$where=spc_common=\''+value+'\'' +\
        '&$group=boroname,health' +\
        '&$order=boroname,health').replace(' ', '%20')
    print(soql_url)
    soql_trees = pd.read_json(soql_url)
    

    soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
            '$select=boroname,health,steward,count(*)' +\
            '&$where=spc_common=\''+value+'\'' +\
            '&$group=boroname,health,steward' +\
            '&$order=boroname,health,steward').replace(' ', '%20')
    soql_trees2 = pd.read_json(soql_url)
    
    return generate_div(soql_trees, soql_trees2)

def generate_div (soql_trees, soql_trees2) :
    fig_health = px.bar(
        soql_trees, 
        x="health",
        y="count", 
        color="health", 
        facet_col="boroname",
        barmode="group",
        category_orders={
            "health": ["Good", "Fair", "Poor"]
        },
        color_discrete_map={
            'Good': '#00CC96',
            'Fair': '#636EFA',
            'Poor': '#EF553B'
        }
    )
    fig_health.update_traces(width=0.8)

    fig_steward = px.bar(
        soql_trees2,
        x="health",
        y="count",
        color="steward",
        facet_col="boroname",
        barmode="group",
        category_orders={
            "health": ["Good","Fair","Poor"],
            "steward": ["None","1or2","3or4","4orMore"]
        },
        color_discrete_map={
            'None': 'rgb(102,102,102)',
            '1or2': 'rgb(166,118,29)',
            '3or4': 'rgb(230,171,2)',
            '4orMore': 'rgb(102,166,30)',
        }
    )
    
    return html.Div(
        children=[
            # 1st Row
            html.Div([
                html.H3(children='Health'),

                dcc.Graph(
                    id='health',
                    figure=fig_health
                ),
            ], className='row'),
            # 2nd Row
            html.Div([
                html.H3(children='Steward'),

                dcc.Graph(
                    id='steward',
                    figure=fig_steward
                ),  
            ], className='row'),
    ])  

if __name__ == '__main__':
    app.run_server(debug=True, mode="external")

/var/folders/7p/8y90x96j6y7693pt7grg64b00000gn/T/ipykernel_78804/2388716980.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/7p/8y90x96j6y7693pt7grg64b00000gn/T/ipykernel_78804/2388716980.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


Dash app running on http://127.0.0.1:8050/
